In [11]:
!pip install crewai crewai_tools boto3


In [2]:
from pydantic import BaseModel , Field
from typing import List

class OptionSchema(BaseModel):
    """Schema representing a possible answer option for a question in a survey."""
    option: str = Field(description="A potential answer choice for the question in the survey.")

class QuestionSchema(BaseModel):
    """Schema representing a survey question, its type, and its possible answer options."""
    question: str = Field(description="The question text, clearly phrased for survey respondents.")
    question_type: str = Field(description="The type of the question. Choose one: binary, rating, multiple choice, or free text.")
    options: List[OptionSchema] = Field(description="A list of potential answer options. Relevant only for questions with answer choices.")

class QuestionsSchema(BaseModel):
    """Schema for representing a collection of questions along with their types and possible answer options."""
    question_config: List[QuestionSchema] = Field(description="A list of questions, each with its type and potential answer options.")


In [35]:
import os
from crewai import Agent, Crew, Process, Task
from crewai_tools import SerperDevTool, WebsiteSearchTool
from crewai import LLM


llm_claude = LLM(
    model="bedrock/us.anthropic.claude-3-opus-20240229-v1:0"
)

search_tool = SerperDevTool()

question_generator = Agent(
    role="Survey Questionnaire Expert",
    goal="Design tailored survey questions with relevant types and options based on specific contexts.",
    backstory="An AI adept at creating professional and targeted survey questions for companies.",
    llm=llm_claude,
    verbose=True
)

context_researcher = Agent(
    role="Context and Audience Research Specialist",
    goal="Extract actionable insights about the company, industry, and audience for survey creation.",
    backstory="An AI proficient in sourcing, analyzing, and summarizing relevant data to enhance survey design.",
    llm=llm_claude,
    tools=[search_tool],
    verbose=True
)

researcher_task = Task(
    description="""
    1. Analyze the provided requirements and extract key themes or focus areas.
    2. Research the company, {company}, its industry, {industry}, and target audience, {audience} using available tools.
    3. Summarize the findings, including:
       - Industry trends and challenges.
       - Key audience characteristics and preferences.
       - Any existing survey data or related insights.
    4. Deliver concise, structured, and actionable insights tailored to the survey generation process.
    """,
    expected_output="A structured set of research insights including industry trends, company specifics, audience profile, and competitor analysis.",
    agent=context_researcher,
)

generation_task = Task(
    description="""
    1. Leverage the insights provided by the researcher to generate a total of {no_of_questions} survey questions relevant to {company} in the {industry} industry, considering the target audience: {audience}..
    2. Classify each question as one of the following types: binary, multi-choice, rating, or free text.
    3. Provide detailed answer options for binary, multi-choice, and rating questions.
    4. Ensure clarity, relevance, and audience alignment in the final output.
    """,
    expected_output="A comprehensive list of survey questions with classifications and answer options.",
    agent=question_generator,
    context=[researcher_task],
    output_pydantic=QuestionsSchema
)

survey_crew = Crew(
    agents=[question_generator, context_researcher],
    tasks=[researcher_task, generation_task],
    verbose=True,
)




LLM value is already an LLM object
LLM value is already an LLM object


In [36]:
result = survey_crew.kickoff(inputs={'no_of_questions': '10','company':"fayredge","industry":"consultation","audience":"customer"})

# Agent: Context and Audience Research Specialist
## Task: 
    1. Analyze the provided requirements and extract key themes or focus areas.
    2. Research the company, fayredge, its industry, consultation, and target audience, customer using available tools.
    3. Summarize the findings, including:
       - Industry trends and challenges.
       - Key audience characteristics and preferences.
       - Any existing survey data or related insights.
    4. Deliver concise, structured, and actionable insights tailored to the survey generation process.
    


# Agent: Context and Audience Research Specialist
## Thought: Thought: To begin, I should search for information about fayredge and its industry to understand the context for the survey.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"fayredge consultation company\"}"
## Tool Output: 
{'searchParameters': {'q': 'fayredge consultation company', 'type': 'search', 'num': 10, 'engine': 'google'}, 'org